In [2]:
from collections import defaultdict, deque

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report,
                             make_scorer, recall_score)
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler

## Manipulating the data
### Loading and cleaning the data with pandas

In [3]:
# Load the datasets with the specified file paths
no_attack_path = "./BATADAL_dataset03.csv"
with_attacks_path = "./BATADAL_dataset04.csv"
attacks_info_path = "./Attacks_TrainingDataset2.csv"

# Read the data into pandas DataFrames
no_attacks = pd.read_csv(no_attack_path)
with_attacks = pd.read_csv(with_attacks_path)
batadal_attacks_info = pd.read_csv(attacks_info_path)

# Strip any leading/trailing spaces in column names
no_attacks.columns = no_attacks.columns.str.strip()
with_attacks.columns = with_attacks.columns.str.strip()

batadal = pd.concat([no_attacks,with_attacks], ignore_index=True)
batadal

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,06/01/14 00,0.509730,2.049003,3.191145,2.792634,2.656091,5.316831,1.562321,98.998444,1.0,...,87.605774,26.495605,84.206619,18.901676,81.983734,18.791777,67.125603,29.387470,28.487471,0
1,06/01/14 01,0.412580,2.009072,3.642565,2.831673,3.126387,5.494855,1.852043,99.095901,1.0,...,89.448341,26.487326,85.900085,18.849329,82.150589,18.739643,67.178696,29.354256,28.454256,0
2,06/01/14 02,0.320112,1.986093,4.140192,3.256733,3.574601,5.500000,2.246126,98.420959,1.0,...,91.056114,26.487364,86.582474,19.597170,83.988579,19.496712,72.425293,29.354538,28.454538,0
3,06/01/14 03,0.332879,2.009203,4.673478,3.744497,3.952379,5.500000,3.203573,97.575172,1.0,...,92.594353,26.575815,88.020546,26.028486,64.670486,25.922703,76.275040,29.449951,28.549952,0
4,06/01/14 04,0.483496,2.089049,5.237937,4.409456,3.504676,5.500000,4.439714,97.351059,1.0,...,94.473099,26.723457,90.422462,26.209970,64.746620,26.104692,76.703529,29.574265,28.674263,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12933,24/12/16 20,2.650000,2.370000,3.850000,3.040000,3.820000,4.940000,2.190000,120.080000,1.0,...,70.030000,27.380000,84.140000,18.450000,81.670000,18.340000,66.040000,29.880000,28.980000,-999
12934,24/12/16 21,2.240000,2.560000,3.420000,2.920000,3.690000,5.020000,1.970000,119.120000,1.0,...,68.600000,27.660000,83.460000,25.400000,60.850000,25.280000,66.890000,30.190000,29.290000,-999
12935,24/12/16 22,1.910000,2.760000,2.950000,2.490000,2.700000,5.140000,1.870000,120.710000,1.0,...,85.630000,26.840000,82.820000,24.460000,59.560000,24.340000,66.080000,29.680000,28.780000,-999
12936,24/12/16 23,1.520000,2.520000,3.330000,2.030000,1.690000,5.100000,1.390000,120.020000,1.0,...,86.150000,25.780000,103.630000,24.770000,59.010000,24.650000,66.420000,28.980000,28.080000,-999


We need to fix the ATT_FLAG column (and all columns for that matter... not taking any chances...)
Some columns have too complex names with special caracters.

In [4]:
# Rename columns to simplify access
batadal_attacks_info.rename(
    columns={
        "Starting time [dd/mm/YY HH]": "Start_Time",
        "Ending time [dd/mm/YY HH]": "End_Time",
    },
    inplace=True,
)

We see that Starting time [dd/mm/YY HH], Ending time [dd/mm/YY HH] and DATETIME columns should be dates instead of strings/objects. We can use pandas to convert this data.

In [5]:
# Convert 'DATETIME' columns to datetime format for easy comparison
batadal["DATETIME"] = pd.to_datetime(
    batadal["DATETIME"],
    format="%d/%m/%y %H",
    dayfirst=True,
    errors="coerce",
)
batadal.iloc[:, [0, 1, 20, 25, 44]].info()

batadal_attacks_info["Start_Time"] = pd.to_datetime(
    batadal_attacks_info["Start_Time"],
    format="%d/%m/%Y %H",
    dayfirst=True,
    errors="coerce",
)
batadal_attacks_info["End_Time"] = pd.to_datetime(
    batadal_attacks_info["End_Time"],
    format="%d/%m/%Y %H",
    dayfirst=True,
    errors="coerce",
)
print("\n")
batadal_attacks_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12938 entries, 0 to 12937
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DATETIME  12938 non-null  datetime64[ns]
 1   L_T1      12938 non-null  float64       
 2   F_PU7     12938 non-null  float64       
 3   S_PU9     12938 non-null  float64       
 4   ATT_FLAG  12938 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 505.5 KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  7 non-null      int64         
 1   Start_Time          7 non-null      datetime64[ns]
 2   End_Time            7 non-null      datetime64[ns]
 3   Duration [hours]    7 non-null      int64         
 4   Attack description  7 non-null      object        
 5   

In [6]:
fix_attack_periods = True

batadal_fixed = batadal.copy()

# Loop through each attack period and set ATT_FLAG and T1_ATT_FLAG to T7_ATT_FLAG
for index, row in batadal_attacks_info.iterrows():
    # Extract start and end of the attack period from the attacks dataset
    attack_start = pd.to_datetime(row["Start_Time"], format="%d/%m/%Y %H")
    attack_end = pd.to_datetime(row["End_Time"], format="%d/%m/%Y %H")

    # Update ATT_FLAG for known attack periods
    if (fix_attack_periods):
        batadal_fixed.loc[
            (batadal_fixed["DATETIME"] >= attack_start)
            & (batadal_fixed["DATETIME"] <= attack_end),
            "ATT_FLAG"
        ] = 1

    # Set ATT_FLAG to 0 for non-attack periods (everything else remaining)
    batadal_fixed.loc[
        (batadal_fixed["ATT_FLAG"] == -999), "ATT_FLAG"
    ] = 0

# Calculate the percentage of rows labeled as attack (ATT_FLAG == 1)
final_attack_count = batadal_fixed[batadal_fixed["ATT_FLAG"] == 1]["ATT_FLAG"].count()
final_attack_percentage = (final_attack_count / len(batadal_fixed)* 100)
print(
    f"Percentage of attack data after cross-referencing: {final_attack_percentage:.2f}%\n"
)
print(
    f"Nb of attack data after cross-referencing: {final_attack_count}\n"
)

batadal_fixed

Percentage of attack data after cross-referencing: 3.80%

Nb of attack data after cross-referencing: 492



,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,2014-01-06 00:00:00,0.509730,2.049003,3.191145,2.792634,2.656091,5.316831,1.562321,98.998444,1.0,...,87.605774,26.495605,84.206619,18.901676,81.983734,18.791777,67.125603,29.387470,28.487471,0
1,2014-01-06 01:00:00,0.412580,2.009072,3.642565,2.831673,3.126387,5.494855,1.852043,99.095901,1.0,...,89.448341,26.487326,85.900085,18.849329,82.150589,18.739643,67.178696,29.354256,28.454256,0
2,2014-01-06 02:00:00,0.320112,1.986093,4.140192,3.256733,3.574601,5.500000,2.246126,98.420959,1.0,...,91.056114,26.487364,86.582474,19.597170,83.988579,19.496712,72.425293,29.354538,28.454538,0
3,2014-01-06 03:00:00,0.332879,2.009203,4.673478,3.744497,3.952379,5.500000,3.203573,97.575172,1.0,...,92.594353,26.575815,88.020546,26.028486,64.670486,25.922703,76.275040,29.449951,28.549952,0
4,2014-01-06 04:00:00,0.483496,2.089049,5.237937,4.409456,3.504676,5.500000,4.439714,97.351059,1.0,...,94.473099,26.723457,90.422462,26.209970,64.746620,26.104692,76.703529,29.574265,28.674263,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12933,2016-12-24 20:00:00,2.650000,2.370000,3.850000,3.040000,3.820000,4.940000,2.190000,120.080000,1.0,...,70.030000,27.380000,84.140000,18.450000,81.670000,18.340000,66.040000,29.880000,28.980000,0
12934,2016-12-24 21:00:00,2.240000,2.560000,3.420000,2.920000,3.690000,5.020000,1.970000,119.120000,1.0,...,68.600000,27.660000,83.460000,25.400000,60.850000,25.280000,66.890000,30.190000,29.290000,0
12935,2016-12-24 22:00:00,1.910000,2.760000,2.950000,2.490000,2.700000,5.140000,1.870000,120.710000,1.0,...,85.630000,26.840000,82.820000,24.460000,59.560000,24.340000,66.080000,29.680000,28.780000,0
12936,2016-12-24 23:00:00,1.520000,2.520000,3.330000,2.030000,1.690000,5.100000,1.390000,120.020000,1.0,...,86.150000,25.780000,103.630000,24.770000,59.010000,24.650000,66.420000,28.980000,28.080000,0


In [10]:
# Step 1: Read the .INP file and construct the graph
file_path = "./CTOWN.INP"
graph = defaultdict(list)

with open(file_path, "r") as file:
    lines = file.readlines()

# Construct the graph from the [PIPES], [CONTROLS], and [JUNCTIONS] sections
in_pipes_section = False
in_controls_section = False
in_junctions_section = False

junctions = set()  # Set to store all junction IDs

for line in lines:
    line = line.strip()

    if line == "[PIPES]":
        in_pipes_section = True
        continue
    if line == "[CONTROLS]":
        in_pipes_section = False
        in_controls_section = True
        continue
    if line == "[JUNCTIONS]":
        in_controls_section = False
        in_junctions_section = True
        continue
    if line.startswith("[") and line not in [
        "[PIPES]",
        "[CONTROLS]",
        "[JUNCTIONS]",
    ]:
        in_pipes_section = False
        in_controls_section = False
        in_junctions_section = False

    # Construct the graph from the [PIPES] section
    if in_pipes_section and line and not line.startswith(";"):
        parts = line.split()
        if len(parts) >= 3:
            node1, node2 = parts[1], parts[2]
            graph[node1].append(node2)
            graph[node2].append(node1)

    # Include pump actuators in the graph based on the [CONTROLS] section
    if in_controls_section and "LINK" in line and "NODE" in line:
        parts = line.split()
        pump = parts[1]  # The pump actuator
        node = parts[4]  # The node controlling the pump
        graph[node].append(pump)
        graph[pump].append(node)

    # Collect junctions from the [JUNCTIONS] section
    if in_junctions_section and line and not line.startswith(";"):
        parts = line.split()
        if len(parts) > 0:
            junction_id = parts[0]
            junctions.add(junction_id)
            graph[junction_id]  # Add junction to the graph as a node

# Step 2: Find all neighbors by level using BFS
def find_neighbors_by_level(graph, start_node):
    visited = set()
    queue = deque([(start_node, 0)])  # (node, level)
    levels = defaultdict(list)

    while queue:
        node, level = queue.popleft()
        if node not in visited:
            visited.add(node)
            levels[level].append(node)
            for neighbor in graph[node]:
                if neighbor not in visited:
                    queue.append((neighbor, level + 1))

    return levels

neighbors_by_level = {}
neighbors_by_level_in_batadal = {}

# Function to check if a component is present in batadal_data_wth_attack columns
def is_component_in_batadal(component, batadal_columns):
    for prefix in prefixes:
        if f"{prefix}{component}" in batadal_columns:
            return True
    return False

for i in range(1,8):
    # Find all neighbors of T1 by level
    neighbors_by_level[f"T{i}"] = find_neighbors_by_level(graph, f"T{i}")

    # Step 3: Check if components are in batadal_data_wth_attack using naming conventions
    prefixes = ["L_", "S_", "F_", "P_"]
    batadal_columns = batadal_fixed.columns

    # Create a DataFrame to show the results
    data = []
    for level, components in neighbors_by_level[f"T{i}"].items():
        for component in components:
            in_batadal = is_component_in_batadal(component, batadal_columns)
            data.append({"Component": component, "Level": level, "In Batadal": in_batadal})

    neighbors_df = pd.DataFrame(data)

    # Filter and display only components present in batadal_data_wth_attack
    neighbors_by_level_in_batadal[f"T{i}"] = neighbors_df[neighbors_df["In Batadal"] == True]

# Display the filtered DataFrame
print(neighbors_by_level_in_batadal)

{'T1':     Component  Level  In Batadal
0          T1      0        True
41       J302     10        True
52       J307     11        True
95        J14     16        True
103      J422     17        True
131        T2     20        True
133      J289     20        True
139      J300     21        True
148      J269     25        True, 'T2':     Component  Level  In Batadal
0          T2      0        True
3        J422      3        True
7        J289      4        True
9        J300      5        True
12        J14      6        True
130        T1     20        True
132      J302     20        True
140      J307     21        True
148      J269     25        True, 'T3':    Component  Level  In Batadal
0         T3      0        True
11      J256      4        True, 'T4':    Component  Level  In Batadal
0         T4      0        True
74      J415     24        True, 'T5':    Component  Level  In Batadal
0         T5      0        True
43      J306     11        True, 'T6':    Compone

In [40]:
# Prepare the data
columns_to_exclude = ["DATETIME", "ATT_FLAG"]

def columns_of_interest(neighbors, batadal):
        columns_of_interest_array = []

        # Define the possible prefixes for different sensor types
        prefixes = ["L_", "S_", "F_", "P_"]

        # List to store selected columns for analysis
        columns_of_interest = [
                "ATT_FLAG",
                "DATETIME",
        ]  # Include the datetime and target column (T1)

        # Loop through each component in neighbors_in_batadal_df
        for _, row in neighbors.iterrows():
                component = row["Component"] #ex J317
                # Check if prefixed versions of the component are present in the DataFrame columns
                for prefix in prefixes:
                        col_name = f"{prefix}{component}"
                        if col_name in batadal.columns:
                                columns_of_interest.append(col_name)

        return columns_of_interest

# Add columns T1_attack to T7_attack based on "Attack description" and "SCADA concealment" columns

# Create T1_attack to T7_attack columns initialized with 0
for i in range(1, 8):
    batadal_attacks_info[f"T{i}_attack"] = 0

initial_batadal_colums = batadal_fixed.columns
batadal_fixed_copy = {}
y = {}

for i in range(1,8):
        batadal_fixed_copy[f"T{i}"] = pd.DataFrame()
        batadal_fixed_copy[f"T{i}"]["ATT_FLAG"] = batadal_fixed["ATT_FLAG"]
        for n in range(1,5):
                for col in [col for col in columns_of_interest(neighbors_by_level_in_batadal[f"T{i}"], batadal_fixed) if col not in columns_to_exclude]:
                        # Create a new column representing the algebraic variation between current and previous row
                        batadal_fixed_copy[f"T{i}"][f"{col}_past_{n}"] = batadal_fixed[col].shift(n)

        # Drop the first row with NaN values due to the shift operation
        batadal_fixed_copy[f"T{i}"] = batadal_fixed_copy[f"T{i}"].dropna()
        
        water_tank_str = f"T{i}"
        batadal_attacks_info[f"T{i}_attack"] = batadal_attacks_info.apply(
        lambda row: (
            1
            if (
                water_tank_str in str(row["Attack description"])
                or water_tank_str in str(row["SCADA concealment"])
            )
            else 0
        ),
        axis=1,
    )
        y[f"T{i}"] = batadal_fixed_copy[f"T{i}"]['ATT_FLAG']

batadal_fixed_copy["T1"]

,ATT_FLAG,L_T1_past_1,P_J302_past_1,P_J307_past_1,P_J14_past_1,P_J422_past_1,L_T2_past_1,P_J289_past_1,P_J300_past_1,P_J269_past_1,...,P_J269_past_3,L_T1_past_4,P_J302_past_4,P_J307_past_4,P_J14_past_4,P_J422_past_4,L_T2_past_4,P_J289_past_4,P_J300_past_4,P_J269_past_4
4,0,0.332879,26.028486,25.922703,29.449951,28.549952,2.009203,26.575815,26.519985,34.274750,...,33.453247,0.509730,18.901676,18.791777,29.387470,28.487471,2.049003,26.495605,26.426495,33.506031
5,0,0.483496,26.209970,26.104692,29.574265,28.674263,2.089049,26.723457,26.671642,34.395435,...,33.818413,0.412580,18.849329,18.739643,29.354256,28.454256,2.009072,26.487326,26.422962,33.453247
6,0,0.791114,30.366247,30.366247,33.749393,32.849392,2.773177,31.443146,31.443146,36.115982,...,34.274750,0.320112,19.597170,19.496712,29.354538,28.454538,1.986093,26.487364,26.427771,33.818413
7,0,1.186589,30.004425,30.004425,31.808870,30.908869,3.536068,29.219223,29.259064,35.517700,...,34.395435,0.332879,26.028486,25.922703,29.449951,28.549952,2.009203,26.575815,26.519985,34.274750
8,0,1.420449,26.536455,26.414701,31.700340,30.800341,3.872926,29.160114,29.200378,34.612518,...,36.115982,0.483496,26.209970,26.104692,29.574265,28.674263,2.089049,26.723457,26.671642,34.395435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12933,0,2.940000,26.100000,26.100000,32.070000,31.170000,1.870000,29.810000,29.810000,23.490000,...,22.860000,4.230000,19.850000,19.740000,28.190000,27.290000,0.690000,25.260000,25.190000,22.760000
12934,0,2.650000,18.450000,18.340000,29.880000,28.980000,2.370000,27.380000,27.420000,21.700000,...,22.510000,3.760000,19.900000,19.790000,29.290000,28.390000,1.010000,26.700000,26.740000,22.860000
12935,0,2.240000,25.400000,25.280000,30.190000,29.290000,2.560000,27.660000,27.710000,22.250000,...,23.490000,3.310000,19.700000,19.600000,29.430000,28.530000,1.360000,26.880000,26.910000,22.510000
12936,0,1.910000,24.460000,24.340000,29.680000,28.780000,2.760000,26.840000,26.770000,21.330000,...,21.700000,2.940000,26.100000,26.100000,32.070000,31.170000,1.870000,29.810000,29.810000,23.490000


In [45]:
import plotly.express as px
from sklearn.manifold import TSNE

# Normaliser les données avant d'utiliser t-SNE
scaler = StandardScaler()

for i in range(1,3):
    X_scaled = scaler.fit_transform(batadal_fixed_copy[f"T{i}"])

    # t-SNE to reduce to 3 dimensions
    tnse = TSNE(n_components=3, random_state=42)
    X_tsne = tnse.fit_transform(X_scaled)

    # Convertir le résultat en DataFrame pour faciliter le traçage
    tsne_df = pd.DataFrame(X_tsne, columns=['Dim1', 'Dim2', 'Dim3'])
    tsne_df['ATT_FLAG'] = y[f"T{i}"].values

    # Tracer les points en 3D, avec couleurs selon ATT_FLAG
    fig = px.scatter_3d(tsne_df, x='Dim1', y='Dim2', z='Dim3', color='ATT_FLAG', 
                        color_discrete_map={0: 'blue', 1: 'red'}, 
                        labels={'ATT_FLAG': 'Attack Flag'}, 
                        title="3D t-SNE Projection of Batadal Data")
    fig.update_traces(marker=dict(size=3))  # Ajuster la taille des points
    fig.update_layout(scene=dict(
                        xaxis_title='Dimension 1',
                        yaxis_title='Dimension 2',
                        zaxis_title='Dimension 3'))

    # Afficher le graphique interactif
    fig.show()

In [44]:
from sklearn.metrics import f1_score

for i in range(1,3):

    # Séparer les cas ATT_FLAG = 1 pour permettre d'entrainer un "modèle statistique sans erreurs / déformations cyber" 
    df = batadal_fixed_copy[f"T{i}"]
    X0 = df[df['ATT_FLAG'] == 0].copy()
    y0 = y[f"T{i}"][y[f"T{i}"] == 0].copy()

    # Normaliser les données avant d'utiliser t-SNE
    scaler = StandardScaler()
    X0_scaled = scaler.fit_transform(X0)
    X_scaled = scaler.fit_transform(batadal_fixed_copy[f"T{i}"])

    # Appliquer l'Isolation Forest
    iso_forest = IsolationForest(contamination=0.01, random_state=10).fit(X0_scaled)  # Ajuster contamination selon le taux d'anomalie attendu

    y_pred = iso_forest.predict(X_scaled)

    # Convertir les résultats en DataFrame pour une meilleure lisibilité
    results_df = pd.DataFrame({
        'Is_Outlier': y_pred,  # -1 = anomalie, 1 = normal
        'ATT_FLAG': y[f"T{i}"].values
    })

    # Calcul du F1-Score basé sur ATT_FLAG comme vérité terrain
    y_pred_binary = (results_df['Is_Outlier'] == -1).astype(int)
    f1 = f1_score(results_df['ATT_FLAG'], y_pred_binary)

    # Calcul des métriques pour ATT_FLAG = 1 et ATT_FLAG = 0
    outliers_att_1 = results_df[(results_df['Is_Outlier'] == -1) & (results_df['ATT_FLAG'] == 1)].shape[0]
    inliers_att_0 = results_df[(results_df['Is_Outlier'] == 1) & (results_df['ATT_FLAG'] == 0)].shape[0]
    inliers_att_1 = results_df[(results_df['Is_Outlier'] == 1) & (results_df['ATT_FLAG'] == 1)].shape[0]
    outliers_att_0 = results_df[(results_df['Is_Outlier'] == -1) & (results_df['ATT_FLAG'] == 0)].shape[0]

    # Affichage des résultats finaux
    print(f"Resultats pour T{i}:\n")
    print(f"Nombre de points avec ATT_FLAG = 1 qui sont hors de la distribution: {outliers_att_1}\n")
    print(f"Nombre de points avec ATT_FLAG = 1 qui sont dans la distribution: {inliers_att_1}\n")
    print(f"Nombre de points avec ATT_FLAG = 0 qui sont dans la distribution: {inliers_att_0}\n")
    print(f"Nombre de points avec ATT_FLAG = 0 qui sont hors de la distribution: {outliers_att_0}\n")
    print(f"F1-Score: {f1:.4f}")

Resultats pour T1:

Nombre de points avec ATT_FLAG = 1 qui sont hors de la distribution: 39

Nombre de points avec ATT_FLAG = 1 qui sont dans la distribution: 453

Nombre de points avec ATT_FLAG = 0 qui sont dans la distribution: 12339

Nombre de points avec ATT_FLAG = 0 qui sont hors de la distribution: 103

F1-Score: 0.1230
Resultats pour T2:

Nombre de points avec ATT_FLAG = 1 qui sont hors de la distribution: 41

Nombre de points avec ATT_FLAG = 1 qui sont dans la distribution: 451

Nombre de points avec ATT_FLAG = 0 qui sont dans la distribution: 12344

Nombre de points avec ATT_FLAG = 0 qui sont hors de la distribution: 98

F1-Score: 0.1300
